In [2]:
from openbabel import openbabel

In [3]:
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [5]:
from ast import literal_eval

In [6]:
from treelib import Node, Tree

In [7]:
class Molecule(object):
    def __init__(self, path):
        self.path = path

In [8]:
def split(word):
    return[char for char in word]

In [9]:
def index_finder(Product, rels, path):
    index = 0
    for i in range(len(rels['Index'])):
        found_index = False
        place = literal_eval(rels['Products'][i])
        for j in range(len(place)):
            if place[j] == Product:
                valid = True
                precursors = precursor_finder(rels['Index'][i], rels)
                for k in range(len(precursors)):
                    if precursors[k] in path:
                        valid = False
                        break
                if valid == True:
                    found_index = True
            if found_index == True:
                index = rels['Index'][i]
                print(rels['Index'][i])
                print(precursors)
                return(index)
                break

In [10]:
def precursor_finder(index, rels):
    precursors = []
    for i in range(len(rels['Index'])):
        if rels['Index'][i] == index:
            dummy = literal_eval(rels['Reagents'][i])
            for j in range(len(dummy)):
                precursors.append(dummy[j])
    return(precursors)

In [15]:
def find_component(node, rels):
    Product = node.tag
    path = (node.data).path
    print(path)
    index = index_finder(Product, rels, path)
    precursors = precursor_finder(index, rels)
    return(precursors)

In [16]:
def map_tree(Smiles, rels):
    rels = pd.read_csv(rels, sep='\t')
    base_molecules = ['N', 'C=O', 'C(CO)=O', 'O'] #FormoseAmm
    #base_molecules = ['C(C(C)=O)(O)=O', 'O'] #PyruvicAcid
    tree = Tree()
    tree.create_node(Smiles, 0, data=Molecule([Smiles]))
    finished = False
    level_counter = 0
    node_counter = 0
    while finished == False:
        print(level_counter)
        nodes = tree.all_nodes()
        active_nodes = []
        for i in range(len(nodes)):
            tag = nodes[i].tag
            level = tree.depth(nodes[i])
            if level == level_counter and tag not in base_molecules:
                active_nodes.append(nodes[i])
        if active_nodes == []:
            finished = True
            break
        else:
            for i in range(len(active_nodes)):
                precursors = find_component(active_nodes[i], rels)
                product = active_nodes[i].identifier
                dummy = (active_nodes[i].data).path
                place = []
                for k in range(len(dummy)):
                    place.append(dummy[k])
                for j in range(len(precursors)):
                    node_counter +=1
                    tree.create_node(precursors[j], node_counter, parent=product, data=Molecule(place + [precursors[j]]))
            level_counter+=1
    return(tree)

In [18]:
%%time
a = map_tree('C(O)=O', './ProcessedRels/FormoseAmmProcessedRels.tsv')

0
['C(O)=O']
13_0
['C=O', 'C=O', 'O']
1
CPU times: user 1.38 s, sys: 40.6 ms, total: 1.42 s
Wall time: 1.43 s


In [19]:
a.show(data_property='path')

['C(O)=O']
├── ['C(O)=O', 'C=O']
├── ['C(O)=O', 'C=O']
└── ['C(O)=O', 'O']



In [81]:
a.show(data_property='path')

['C(C(CO)O)=O']
├── ['C(C(CO)O)=O', 'C(CO)=O']
└── ['C(C(CO)O)=O', 'C=O']

